# Лабораторная работа №7. "Полносвязные нейронные сети (многослойный персептрон). Решение задач регрессии и классификации"

Задание №1.
Решить задачи регрессии и классификации на данных в соответствии с Вашим индивидуальным вариантом (см. Лаб.работы №3, 4), используя полносвязные НС; реализовать НС посредством API Keras и фреймворка TensorFlow; оценить качество полученных моделей с помощью метрик.

Задание №2.
Разработать многослойный персептрон (MLP), с помощью которого можно решать задачи регрессии и классификации. Предусмотреть возможность использования таких функции активации, как sigmoid, tanh и relu; также предусмотреть возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой. Реализовать обучение MLP методом обратного распространения ошибки; самостоятельно найти производные функций sigmoid, tanh и relu; реализовать классический градиентный спуск с возможностью указания шага.


Дополнительное Задание №3*.
1. Самостоятельно изучить отличия работы оптимизаторов Adam и RMSProp от классического градиентного спуска.
2. Реализовать градиентный спуск с использованием указанных оптимизаторов; предусмотрите возможность использования реализованных вами оптимизаторов в Вашем персептроне.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler
#from sklearn.ensemble import RandomForestRegressor

In [3]:
data_regression = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Машинное обучение 2 курс/Лабораторная работа №7/data_reg_final.csv")
data_classification = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Машинное обучение 2 курс/Лабораторная работа №7/data_cl.csv')

In [4]:
X_regression = data_regression.drop('Price', axis=1)
y_regression = data_regression["Price"]

X_classification = data_classification.drop('Fire Alarm', axis=1)
y_classification = data_classification['Fire Alarm']

In [ ]:
# clf = RandomForestRegressor()
# clf.fit(X_regression_train, y_regression_train)

In [ ]:
# importances_f = pd.Series(clf.feature_importances_, index = X_regression_train.columns)
# top_f = importances_f.nlargest(30).index

In [ ]:
# top_list = list(top_f)

In [ ]:
# top_list.append('Price')

In [ ]:
# data_regression.loc[:, top_list].to_csv('data_reg_final.csv',index=False)

In [ ]:
data_regression.head()

,Year,Age,Transmission_Automatic,Engine_capacity,Distance,Transmission_Manual,Style_Crossover,Fuel_type_Diesel,Style_SUV,Make_Toyota,...,Make_Audi,Fuel_type_Metan/Propan,Model_M Class,Make_Hyundai,Model_Superb,Make_Honda,Model_GLC,Model_A6,Make_Ford,Price
0,2011.0,12.0,1.0,1800.0,195000.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7750.0
1,2014.0,9.0,0.0,1500.0,135000.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8550.0
2,2012.0,11.0,0.0,1500.0,110000.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6550.0
3,2006.0,17.0,0.0,1600.0,200000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4100.0
4,2000.0,23.0,0.0,2200.0,300000.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3490.0


In [ ]:
data_classification.head()

,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],Fire Alarm
0,17.557,53.23,1109.0,490.0,12849.0,19453,938.905,Yes
1,21.330,47.26,132.0,400.0,13216.0,20085,939.646,Yes
2,12.206,53.22,1153.0,408.0,12878.0,19437,938.757,Yes
3,26.510,49.45,1289.0,400.0,12960.0,19405,938.688,Yes
4,19.910,53.67,124.0,400.0,13220.0,20098,939.656,Yes


In [ ]:
data_regression.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34040 entries, 0 to 34039
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Year                      34040 non-null  float64
 1   Age                       34040 non-null  float64
 2   Transmission_Automatic    34040 non-null  float64
 3   Engine_capacity           34040 non-null  float64
 4   Distance                  34040 non-null  float64
 5   Transmission_Manual       34040 non-null  float64
 6   Style_Crossover           34040 non-null  float64
 7   Fuel_type_Diesel          34040 non-null  float64
 8   Style_SUV                 34040 non-null  float64
 9   Make_Toyota               34040 non-null  float64
 10  Fuel_type_Petrol          34040 non-null  float64
 11  Model_5 Series            34040 non-null  float64
 12  Model_X5                  34040 non-null  float64
 13  Make_Mercedes             34040 non-null  float64
 14  Make_B

In [ ]:
data_classification.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61161 entries, 0 to 61160
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Temperature[C]  61161 non-null  float64
 1   Humidity[%]     61161 non-null  float64
 2   TVOC[ppb]       61161 non-null  float64
 3   eCO2[ppm]       61161 non-null  float64
 4   Raw H2          61161 non-null  float64
 5   Raw Ethanol     61161 non-null  int64  
 6   Pressure[hPa]   61161 non-null  float64
 7   Fire Alarm      61161 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 3.7+ MB


In [ ]:
data_classification['Fire Alarm'] = data_classification['Fire Alarm'].map({"Yes": 1, "No": 0})

In [5]:
X_regression_train, X_regression_test, y_regression_train, y_regression_test = train_test_split(X_regression,
                                                                                                y_regression,
                                                                                                test_size=0.2)
X_classification_train, X_classification_test, y_classification_train, y_classification_test = train_test_split(X_classification,
                                                                                                                y_classification,
                                                                                                                stratify=y_classification,
                                                                                                                test_size=0.2)

# Решение задачи регрессии

In [6]:
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(1024, activation="relu", input_shape=(30,)),
        tf.keras.layers.Dense(256, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [ ]:
model_regression.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              31744     
                                                                 
 dense_1 (Dense)             (None, 256)               262400    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 310657 (1.19 MB)
Trainable params: 310657 

In [7]:
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss="mse")

In [8]:
model_regression.fit(X_regression_train, y_regression_train, epochs=30, batch_size=64)

Epoch 1/30
426/426 [==============================] - 8s 3ms/step - loss: 40309804.0000
Epoch 2/30
426/426 [==============================] - 1s 3ms/step - loss: 26573864.0000
Epoch 3/30
426/426 [==============================] - 1s 3ms/step - loss: 25354686.0000
Epoch 4/30
426/426 [==============================] - 2s 4ms/step - loss: 25133864.0000
Epoch 5/30
426/426 [==============================] - 2s 5ms/step - loss: 24320736.0000
Epoch 6/30
426/426 [==============================] - 2s 5ms/step - loss: 23843028.0000
Epoch 7/30
426/426 [==============================] - 1s 3ms/step - loss: 23722478.0000
Epoch 8/30
426/426 [==============================] - 1s 3ms/step - loss: 23695798.0000
Epoch 9/30
426/426 [==============================] - 1s 3ms/step - loss: 23072402.0000
Epoch 10/30
426/426 [==============================] - 1s 3ms/step - loss: 22048286.0000
Epoch 11/30
426/426 [==============================] - 1s 3ms/step - loss: 21303378.0000
Epoch 12/30
426/426 [=========

In [ ]:
print(mean_absolute_error(y_regression_test, model_regression.predict(X_regression_test)))
print(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test)))

213/213 [==============================] - 0s 2ms/step
2691.5178514025606
213/213 [==============================] - 0s 2ms/step
13316051.452174347


In [ ]:
np.sqrt(12252491.600945756) # RMSE на тесте

3500.3559248947463

# Решение задачи классификации

In [ ]:
X_classification.head()

,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa]
0,17.557,53.23,1109.0,490.0,12849.0,19453,938.905
1,21.330,47.26,132.0,400.0,13216.0,20085,939.646
2,12.206,53.22,1153.0,408.0,12878.0,19437,938.757
3,26.510,49.45,1289.0,400.0,12960.0,19405,938.688
4,19.910,53.67,124.0,400.0,13220.0,20098,939.656


In [ ]:
X_classification.shape

(61161, 7)

In [ ]:
model_cl = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(32, activation="relu", input_shape=(7,)),
        tf.keras.layers.Dense(16, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)

In [ ]:
model_cl.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss="sparse_categorical_crossentropy")

In [ ]:
model_cl.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                256       
                                                                 
 dense_5 (Dense)             (None, 16)                528       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_6 (Dense)             (None, 8)                 136       
                                                                 
 dropout_3 (Dropout)         (None, 8)                 0         
                                                                 
 dense_7 (Dense)             (None, 2)                 18        
                                                                 
Total params: 938 (3.66 KB)
Trainable params: 938 (3.6

In [ ]:
model_cl.fit(X_classification_train, y_classification_train, epochs=30, batch_size=64)

In [ ]:
y_pred_prob = model_cl.predict(X_classification_test)
y_pred = tf.argmax(y_pred_prob, axis=1)

383/383 [==============================] - 2s 4ms/step


In [ ]:
print(accuracy_score(y_classification_test, y_pred))
print(f1_score(y_classification_test, y_pred))

0.9218507316275648
0.9478564415839424


# Задание 2

In [ ]:
class MLP_regression:
    def __init__(self, layer_sizes, activation_functions):
        self.layer_sizes = layer_sizes
        self.activation_functions = activation_functions
        self.weights = []
        self.biases = []
        self.activations = []
        for i in range(len(layer_sizes) - 1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]))
            self.biases.append(np.random.randn(layer_sizes[i + 1]))
            self.activations.append(np.zeros(layer_sizes[i + 1]))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return np.tanh(x)

    def relu(self, x):
        return np.maximum(0, x)

    def derivative(self, x, activation_function):
        if activation_function == 'sigmoid':
            return x * (1 - x)
        elif activation_function == 'tanh':
            return 1 - np.power(x, 2)
        elif activation_function == 'relu':
            return (x > 0).astype(float)

    def forward(self, x):
        for i in range(len(self.weights)):
            x = np.dot(x, self.weights[i]) + self.biases[i]
            if self.activation_functions[i] == 'sigmoid':
                x = self.sigmoid(x)
            elif self.activation_functions[i] == 'tanh':
                x = self.tanh(x)
            elif self.activation_functions[i] == 'relu':
                x = self.relu(x)
            self.activations[i] = x
        return x

    def backward(self, x, y, learning_rate):
        output = self.forward(x)
        deltas = []
        for i in reversed(range(len(self.weights))):
            if i == len(self.weights) - 1:
                error = output - y
            else:
                error = np.dot(deltas[-1], self.weights[i + 1].T)
            delta = error * self.derivative(self.activations[i], self.activation_functions[i])
            deltas.append(delta)
        deltas.reverse()
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * np.dot(x.T if i == 0 else self.activations[i - 1].T, deltas[i])
            self.biases[i] -= learning_rate * np.sum(deltas[i], axis=0)

    def train(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            self.backward(X, y, learning_rate)

    def r2_score(self, y_true, y_pred):
        ss_res = np.sum(np.square(y_true - y_pred))
        ss_tot = np.sum(np.square(y_true - np.mean(y_true)))
        return 1 - ss_res / ss_tot

In [ ]:
class MLP_clasification:
    def __init__(self, layer_sizes, activation_functions):
        self.layer_sizes = layer_sizes
        self.activation_functions = activation_functions
        self.weights = [np.random.randn(y, x) for x, y in zip(layer_sizes[:-1], layer_sizes[1:])]
        self.biases = [np.random.randn(y, 1) for y in layer_sizes[1:]]

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def tanh(self, z):
        return np.tanh(z)

    def tanh_derivative(self, z):
        return 1 - np.tanh(z) ** 2

    def relu(self, z):
        return np.maximum(0, z)

    def relu_derivative(self, z):
        return (z > 0) * 1

    def forward_propagation(self, input_data):
        for b, w, activation in zip(self.biases, self.weights, self.activation_functions):
            if activation == 'sigmoid':
                input_data = self.sigmoid(np.dot(w, input_data) + b)
            elif activation == 'tanh':
                input_data = self.tanh(np.dot(w, input_data) + b)
            elif activation == 'relu':
                input_data = self.relu(np.dot(w, input_data) + b)
        return input_data

    def backpropagation(self, input_data, target):
        gradient_b = [np.zeros(b.shape) for b in self.biases]
        gradient_w = [np.zeros(w.shape) for w in self.weights]

        # Forward pass
        activation = input_data
        activations = [input_data]
        zs = []

        for b, w, activation_function in zip(self.biases, self.weights, self.activation_functions):
            z = np.dot(w, activation) + b
            zs.append(z)
            if activation_function == 'sigmoid':
                activation = self.sigmoid(z)
            elif activation_function == 'tanh':
                activation = self.tanh(z)
            elif activation_function == 'relu':
                activation = self.relu(z)
            activations.append(activation)

        # Backward pass
        delta = self.cost_derivative(activations[-1], target) * self.get_activation_derivative(activations[-1], self.activation_functions[-1])
        gradient_b[-1] = delta
        gradient_w[-1] = np.dot(delta, activations[-2].T)

        for l in range(2, len(self.layer_sizes)):
            z = zs[-l]
            activation_derivative = self.get_activation_derivative(z, self.activation_functions[-l])
            delta = np.dot(self.weights[-l + 1].T, delta) * activation_derivative
            gradient_b[-l] = delta
            gradient_w[-l] = np.dot(delta, activations[-l - 1].T)

        return gradient_b, gradient_w

    def get_activation_derivative(self, z, activation_function):
        if activation_function == 'sigmoid':
            return self.sigmoid_derivative(z)
        elif activation_function == 'tanh':
            return self.tanh_derivative(z)
        elif activation_function == 'relu':
            return self.relu_derivative(z)

    def cost_derivative(self, output_activations, target):
        return output_activations - target

    def update_parameters(self, mini_batch, learning_rate):
        sum_gradient_b = [np.zeros(b.shape) for b in self.biases]
        sum_gradient_w = [np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            delta_gradient_b, delta_gradient_w = self.backpropagation(x, y)
            sum_gradient_b = [nb + dnb for nb, dnb in zip(sum_gradient_b, delta_gradient_b)]
            sum_gradient_w = [nw + dnw for nw, dnw in zip(sum_gradient_w, delta_gradient_w)]

        self.weights = [w - (learning_rate / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, sum_gradient_w)]
        self.biases = [b - (learning_rate / len(mini_batch)) * nb
                       for b, nb in zip(self.biases, sum_gradient_b)]

# Классификация

In [ ]:
X = data_classification.drop(['Fire Alarm'], axis=1)
y = data_classification['Fire Alarm'].values.reshape(-1, 1)

In [ ]:
# Нормализация данных
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Разбиение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Подготовка данных для MLP
X_train = [X_train[i].reshape(-1, 1) for i in range(X_train.shape[0])]
y_train = [y_train[i].reshape(-1, 1) for i in range(y_train.shape[0])]
training_data = list(zip(X_train, y_train))

In [ ]:
# Определение параметров сети
input_size = X.shape[1]
hidden_layers = [10, 5]
output_size = 1

In [ ]:
# Создание и обучение MLP
mlp = MLP_clasification([input_size] + hidden_layers + [output_size], ['relu'] * len(hidden_layers) + ['sigmoid'])
learning_rate = 0.01
for epoch in range(20):
    np.random.shuffle(training_data)
    mini_batches = [training_data[k:k+10] for k in range(0, len(training_data), 10)]
    for mini_batch in mini_batches:
        mlp.update_parameters(mini_batch, learning_rate)

In [ ]:
# Функция для получения предсказаний от MLP
def predict(mlp, X):
    predictions = [mlp.forward_propagation(x.reshape(-1, 1)) for x in X]
    return np.array(predictions).squeeze()

In [ ]:
# Получение и оценка предсказаний
predictions = predict(mlp, X_test)
predictions_binary = (predictions > 0.5).astype(int)
accuracy = accuracy_score(y_test, predictions_binary)

print("Accuracy:", accuracy)

Accuracy: 0.9084443717812475


# Регрессия

In [ ]:
data_cleaned = data_regression

q_low = data_cleaned['Price'].quantile(0.01)
q_hi  = data_cleaned['Price'].quantile(0.99)

data_cleaned = data_cleaned[(data_cleaned['Price'] > q_low) & (data_cleaned['Price'] < q_hi)]
X = data_cleaned.drop(columns=['Price']).to_numpy()
y = data_cleaned['Price'].to_numpy().reshape(-1, 1)
X = (X - X.mean(axis=0)) / X.std(axis=0)
y = (y - y.mean()) / y.std()

# Инициализация и обучение модели
layer_sizes = [X.shape[1], 10, 5, 1]  # Примерная архитектура сети
activation_functions = ['tanh', 'relu', 'tanh']
mlp = MLP_regression(layer_sizes, activation_functions)
mlp.train(X, y, learning_rate=0.01, epochs=100)

In [ ]:
y_pred = mlp.forward(X)
r2 = mlp.r2_score(y, y_pred)
print(f"r2_score: {r2}")

r2_score: 0.3000493528347348


# Задание 3

In [ ]:
import numpy as np

# Определение персептрона
class Perceptron:
    def __init__(self, input_size, learning_rate=0.01):
        self.weights = np.random.rand(input_size + 1)
        self.learning_rate = learning_rate

    def predict(self, inputs):
        summation = np.dot(inputs, self.weights[1:]) + self.weights[0]
        return 1 if summation > 0 else 0

    def train(self, training_inputs, labels, optimizer):
        for inputs, label in zip(training_inputs, labels):
            prediction = self.predict(inputs)
            error = label - prediction
            self.weights[1:] += self.learning_rate * error * inputs
            self.weights[0] += self.learning_rate * error
            optimizer.update(self.weights, error, inputs)

# Определение оптимизаторов
class SGD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate

    def update(self, weights, error, inputs):
        gradient = error * inputs
        weights[1:] -= self.learning_rate * gradient
        weights[0] -= self.learning_rate * error

class Adam:
    def __init__(self, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = 0
        self.v = 0
        self.t = 0

    def update(self, weights, error, inputs):
        self.t += 1
        gradient = error * inputs
        self.m = self.beta1 * self.m + (1 - self.beta1) * gradient
        self.v = self.beta2 * self.v + (1 - self.beta2) * (gradient ** 2)
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)
        weights[1:] -= self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)
        weights[0] -= self.learning_rate * error

class RMSprop:
    def __init__(self, learning_rate=0.01, decay_rate=0.9, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.epsilon = epsilon
        self.average_squared_gradients = 0

    def update(self, weights, error, inputs):
        gradient = error * inputs
        self.average_squared_gradients = self.decay_rate * self.average_squared_gradients + (1 - self.decay_rate) * (gradient ** 2)
        weights[1:] -= self.learning_rate * gradient / (np.sqrt(self.average_squared_gradients) + self.epsilon)
        weights[0] -= self.learning_rate * error

# Применение

In [ ]:
input_size = 2  # Пример для двух входных переменных
perceptron = Perceptron(input_size)
sgd_optimizer = SGD(learning_rate=0.01)
adam_optimizer = Adam(learning_rate=0.01)
rmsprop_optimizer = RMSprop(learning_rate=0.01)

In [ ]:
training_inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
labels = np.array([0, 1, 1, 1])

### SGD

In [ ]:
for _ in range(10):
    perceptron.train(training_inputs, labels, sgd_optimizer)
predictions_sgd = [perceptron.predict(x) for x in training_inputs]

### Adam

In [ ]:
for _ in range(10):
    perceptron.train(training_inputs, labels, adam_optimizer)
predictions_adam = [perceptron.predict(x) for x in training_inputs]

### RMSProp

In [ ]:
for _ in range(10):
   perceptron.train(training_inputs, labels, rmsprop_optimizer)
predictions_rmsprop = [perceptron.predict(x) for x in training_inputs]

### Предсказания

In [ ]:
print(f"Предсказания SGD: {predictions_sgd}")

Предсказания SGD: [1, 1, 1, 1]


In [ ]:
print(f"Предсказания Adam: {predictions_adam}")

Предсказания Adam: [1, 1, 1, 1]


In [ ]:
print(f"Предсказания RMSProp: {predictions_rmsprop}")

Предсказания Adam: [1, 1, 1, 1]
